In [1]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import pymongo
from datetime import datetime

In [2]:
def scrape_tweets(keyword, start_date, end_date, tweet_limit):
    
    tweets_list = []
    query = f"{keyword} since:{start_date} until:{end_date}"
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
        if i >= tweet_limit:
            break
        tweets_list.append([tweet.date, tweet.id, tweet.url, tweet.content, tweet.user.username,
                            tweet.replyCount, tweet.retweetCount, tweet.lang, tweet.source, tweet.likeCount])
    df = pd.DataFrame(tweets_list, columns=['Date', 'ID', 'URL', 'Content', 'User', 'Reply Count',
                                            'Retweet Count', 'Language', 'Source', 'Like Count'])
    return df


In [3]:
def store_data_mongodb(keyword, scraped_data):
   client = pymongo.MongoClient("mongodb://localhost:27017/")
   db = client["twitter_data"]
   collection = db[keyword]
   collection.insert_many(scraped_data.to_dict('records'))


In [4]:
import streamlit as st
def app():
    # Set the title
    st.title("Twitter Scraper")
    # Get the keyword input from the user
    keyword = st.text_input("Enter a keyword or hashtag to search:")
    
    # Get the start and end date range input from the user
    start_date = st.date_input("Enter the start date:")
    end_date = st.date_input("Enter the end date:")
    
    # Get the tweet limit input from the user
    tweet_limit = st.slider("Enter the number of tweets to scrape:", min_value=100, max_value=1000, step=100)
    
    # Display a progress bar while scraping the tweets
    progress_bar = st.progress(0)
    
    # Scrape the tweets
    scraped_data = scrape_tweets(keyword, start_date, end_date, tweet_limit)
    
    # Update the progress bar
    progress_bar.progress(100)
    
    # Display the scraped data in a table
    st.write(scraped_data)
    
    # Add a button to store the data into MongoDB
    if st.button("Store Data into MongoDB"):
        store_data_mongodb(keyword, scraped_data)
        st.success("Data stored successfully!")
    
    # Add a button to download the data into CSV format
    if st.button("Download Data as CSV"):
        csv = scraped_data.to_csv(index=False)
        st.download_button( data,label="Download CSV")
